In [1]:
%pip install openpyxl
import pandas as pd
file = "/Users/k.abhishek/Documents/customer_clusters/customer_onboard_data.xlsx"
df = pd.read_excel(file)
print(df.columns)

Note: you may need to restart the kernel to use updated packages.


KeyboardInterrupt: 

In [8]:
len(df)

1048575

In [4]:
import csv
unique_legal_cluster_ids = {}
with open('export.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    csvFile = list(csvFile)
    for arr in csvFile:
        # print(arr)
        
        legal_cluster_id = arr[1]
        trading_name = arr[4].lower()
        country_code = arr[15].lower()
        country_name = arr[16].lower()
        if unique_legal_cluster_ids.get(legal_cluster_id):
            unique_legal_cluster_ids[legal_cluster_id].add((trading_name, country_code, country_name))
        else:
            unique_legal_cluster_ids[legal_cluster_id] = {(trading_name, country_code, country_name)}


In [5]:
print(len(unique_legal_cluster_ids))

949236


In [6]:
count = 0
max_display_count =25
for legal_cluster_id, trading_names in unique_legal_cluster_ids.items():
    if count == max_display_count:
        break
    count += 1
    print(legal_cluster_id, trading_names)

LEGAL_CLUSTER_ID {('trading_name', 'country_code', 'country_name')}
49619936-45e3-3885-a987-e7a8e7efdbd4 {('societe gbetinde sarl', 'bj', 'benin')}
96453282-1268-31d0-a5fa-3a99e1eb897d {('daniel o ekeoma', 'bj', 'benin')}
a9875291-05fc-3258-b577-1c222c2508ed {('goni hala yunus', 'bj', 'benin')}
3b09e743-09ec-3654-b922-cfa8e320adfe {('obawil distribution sarl', 'bj', 'benin')}
fc861d1b-173c-3e84-98ef-056661dfa4f3 {('hinhamin sarl', 'bj', 'benin')}
447da958-6fa4-39a2-9b2d-7e8f30e64631 {('fadonougbo dossou bienvenu', 'bj', 'benin')}
3b873ef1-eac2-31b9-9857-3a434b593e9b {('idohou madjid', 'bj', 'benin')}
5ccde75e-b124-3e54-85ec-861e87718af0 {('gnakk (wa) ltd', 'ng', 'nigeria'), ('gnakk (wa) ltd', 'bj', 'benin'), ('gnakk ltd', 'bj', 'benin')}
cd9e8e94-1a13-32a5-a6cd-aacaccc3e1a4 {('xu zhong fen', 'bj', 'benin')}
b0e9a5d7-6bb0-3000-84e1-11d3845acb6c {('vincee venture nig', 'bj', 'benin')}
43b3d0e8-85d3-34d3-833a-43769cb96c89 {('chalenges bj (cbj) sarl', 'bj', 'benin')}
4c15e651-48d1-3508-958

In [6]:
print(len(unique_legal_cluster_ids))

949236


In [7]:
different_countries = {}
for legal_cluster_id, company_info in unique_legal_cluster_ids.items():
    company_set = set()
    for company in company_info:
        company_set.add(company[1])
    if len(company_set) > 1:
        different_countries[legal_cluster_id] = company_info
    

In [12]:
len(different_countries)

3052

In [15]:
len(different_countries)
different_countries_list = {}
for legal_cluster_id, trading_names in different_countries.items():
    different_countries_list[legal_cluster_id] = list(trading_names) 
import json
print(different_countries_list)
with open('legal_cluster_different_countries.json', 'w') as f:
    json.dump(different_countries_list, f)
    f.close()

{'5ccde75e-b124-3e54-85ec-861e87718af0': [('gnakk (wa) ltd', 'ng', 'nigeria'), ('gnakk (wa) ltd', 'bj', 'benin'), ('gnakk ltd', 'bj', 'benin')], '4c15e651-48d1-3508-958e-77f11f61d695': [('abimbola rashidat dickson', 'bj', 'benin'), ('abimbola rashidat dickson', 'tg', 'togo'), ('dickson abimbola rashidat', 'bj', 'benin')], 'f7029078-ddfb-3a28-9e50-f41026757d2c': [('venus-nina nig ltd', 'ng', 'nigeria'), ('venus nina co', 'ng', 'nigeria'), ('venus nina co', 'bj', 'benin')], 'a284127d-e22f-3ffc-b47e-4f9542c77fe5': [('mercade merchandise nig ltd', 'ng', 'nigeria'), ('mercade merchandise nig ltd', 'bj', 'benin')], '21276d77-1e00-33bf-a695-a05dda59bac7': [('m b automobile services limited', 'ng', 'nigeria'), ('m b automobile services limited', 'bj', 'benin')], 'ba743f18-333b-336c-ab81-8c7ae39c5cf9': [('kange soukpe ekoue', 'bj', 'benin'), ('kange soukpe ekoue', 'tg', 'togo')], '1851e095-6287-33b3-9421-948d932ef662': [('weststar associates ltd', 'ng', 'nigeria'), ('weststar associates ltd', '

In [37]:
unique_legal_cluster_ids_list = {}
for legal_cluster_id, trading_names in unique_legal_cluster_ids.items():
    unique_legal_cluster_ids_list[legal_cluster_id] = list(trading_names) 
import json

with open('unique_legal_cluster_ids.json', 'w') as f:
    json.dump(unique_legal_cluster_ids_list, f)
    f.close()

# Legal level matches based on two criteria:
1. Name of the customer
2. Country 

#Steps
For comparion purposes only:
1. lowercase the names and country codes
2. Remove special characters from the names
3. Remove spaces from the names
4. Remove stop words from the names

Sort the names and do fuzzy matching with the names.

In [2]:
import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'sa', 'company', 'corporation', 'limited'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def clean_company_name(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)
    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

In [3]:
import csv

with open('export.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    data = list(csvFile)
# data = data[:1000]
corpus_ratio = 0.9
num_customer_corpus = int(len(data) * corpus_ratio)
testing_cusomter_corpus = []
existing_legal_cluster_ids = set()
existing_trading_name = {}

corpus_country_hierarchy = {}

for idx, arr in enumerate(data[1:]):
    # print(arr)
    legal_cluster_id = arr[1]
    trading_name = arr[4].lower()
    country_code = arr[15].lower()
    country_name = arr[16].lower()
    if idx < num_customer_corpus:
        existing_legal_cluster_ids.add(legal_cluster_id)
        # cleaning the trading name for comparison
        clean_trading_name = clean_company_name(trading_name)
        existing_trading_name[clean_trading_name] = (legal_cluster_id, trading_name, country_code, country_name)
        if corpus_country_hierarchy.get(country_code):
            corpus_country_hierarchy[country_code][clean_trading_name] = (legal_cluster_id, trading_name, country_code, country_name)
        else:
            corpus_country_hierarchy[country_code] = {clean_trading_name: (legal_cluster_id, trading_name, country_code, country_name)}
    
    else:
        company_info = (legal_cluster_id, trading_name, country_code, country_name)
        testing_cusomter_corpus.append(company_info)


In [4]:
len(existing_trading_name)

879965

In [5]:
len(testing_cusomter_corpus)

104857

In [6]:
from rapidfuzz import fuzz, process
import heapq
def sorted_tokens(name):
    return " ".join(sorted(name.split()))

#calculates the minimum number of edits (insertions, deletions, substitutions) needed to transform one string into another.
def levenshtein_matching_score(input_name, name):
    levenshtein_matching_score = fuzz.ratio(input_name, name)
    return levenshtein_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_set_matching_score(input_name, name):
    token_set_matching_score = fuzz.token_set_ratio(input_name, name)
    return token_set_matching_score

#focus on finding matching subsequences within strings. This is beneficial when comparing strings with extra content or typos.
def partial_matching_score(input_name, name):
    partial_matching_score = fuzz.partial_ratio(input_name, name)
    return partial_matching_score

#focus on finding matching subsequences within strings. This is beneficial when comparing strings with extra content or typos.
def substring_matching_score(input_name, name):
    substring_matching_score = fuzz.partial_ratio(input_name, name)
    return substring_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_sort_ratio_score(input_name, name):
    score = fuzz.token_sort_ratio(input_name, name)
    return score
# Matching
max_number_matches = 5
def match_name(input_name, corpus):
    input_name = clean_company_name(input_name)
    best_match = None
    best_score = 0
    top_matches = []
    heapq.heapify(top_matches)
    similarity_measures = { 
        "levenshtein_matching_score" : (1, levenshtein_matching_score),
        "token_set_matching_score" : (1, token_set_matching_score),
        "substring_matching_score" : (1, substring_matching_score),
        "partial_matching_score" : (1, partial_matching_score),
        "token_sort_ratio_score" : (1, token_sort_ratio_score)
    }
    for name in corpus:
        num_measures = 0
        total_score = 0
        for function_check in similarity_measures.values():
            check, func = function_check
            if check:
                score = func(input_name, name)
                total_score += score
                num_measures +=1
            if num_measures == 0:
                combined_score = 0
            else:
                combined_score = total_score / num_measures
        if len(top_matches) < max_number_matches:
            heapq.heappush(top_matches, (combined_score, name))
        else:
            heapq.heappushpop(top_matches, (combined_score, name))
        if combined_score > best_score:
            best_score = combined_score
            best_match = name
    
    return best_match, best_score, top_matches



In [15]:
from tqdm import tqdm
corpus = list(existing_trading_name.keys())
correct = 0
incorrect = 0
not_applicable = 0
result = {}
result_tuple = {}

def reformat_top_matches(top_matches):
    matches = {}
    sorted_matches = sorted(top_matches, key=lambda x: x[0], reverse=True)
    rank = 1
    for idx, match in enumerate(sorted_matches):
        score = match[0]
        clean_trading_name = match[1]
        trading_name = existing_trading_name[clean_trading_name][1]
        legal_cluster_id = existing_trading_name[clean_trading_name][0]
        match_info = {
            "score" : score,
            "clean_trading_name" : clean_trading_name,
            "trading_name" : trading_name,
            "legal_cluster_id" : legal_cluster_id
        }
        flag = False
        for r, existing_match in matches.items():
            if existing_match[0]["legal_cluster_id"] == legal_cluster_id:
                matches[r].append(match_info)
                flag = True
                break
        if not flag:
            matches[rank] = [match_info]
            rank += 1

    return matches
    
for test_customer in tqdm(testing_cusomter_corpus):
    legal_cluster_id, trading_name, country_code, country_name = test_customer
    clean_trading_name = clean_company_name(trading_name)
    # reducing the search range by country code
    if legal_cluster_id in existing_legal_cluster_ids and corpus_country_hierarchy.get(country_code):
        country_wise_corpus = corpus_country_hierarchy[country_code]
        corpus = list(country_wise_corpus.keys())
        best_match, best_score, top_matches = match_name(clean_trading_name, corpus)
        matched_company_info = existing_trading_name.get(best_match)
        # print(trading_name)
        # print( best_match, best_score)


        # result[trading_name] = reformat_top_matches(top_matches)
        reformatted_top_matches = reformat_top_matches(top_matches)
        # if legal_cluster_id == matched_company_info[0]:
        #     correct +=1
        # else:
        #     incorrect +=1
        if reformatted_top_matches.get(1):
            if legal_cluster_id == reformatted_top_matches[1][0]["legal_cluster_id"]:
                correct +=1
            else:
                incorrect +=1
        str(test_customer)
        result[str(test_customer)] = reformat_top_matches(top_matches)
        
    else:
        not_applicable +=1


100%|██████████| 104857/104857 [36:08<00:00, 48.35it/s]  


"levenshtein_matching_score" : (1, levenshtein_matching_score)
"token_set_matching_score" : (1, token_set_matching_score)

correct, incorrect, not applicable = (724, 462, 9299)


(1, 1, 1, 1, 1)
(6407, 3575, 94875)
0.6418553396113004


(0, 0, 0,)

Table


In [16]:
correct, incorrect, not_applicable

(6428, 3554, 94875)

In [17]:
correct/(correct + incorrect)

0.6439591264275696

In [19]:
string_value = "('7b4f8ac1-0260-3331-baab-6711f756e602', 's s corporation', 'bd', 'bangladesh')"
tuple_value = eval(string_value)
print(tuple_value)

('7b4f8ac1-0260-3331-baab-6711f756e602', 's s corporation', 'bd', 'bangladesh')


In [20]:
import ast
scores = []
verdict = []
for key, top_matches in result.items():
    test_customer = eval(key)
    top_match = top_matches.get(1)[0]
    scores.append(top_match["score"])
    verdict.append(test_customer[0] == top_match["legal_cluster_id"])





In [25]:
print(scores[:10], verdict[:10])

[100.0, 97.64705882352942, 85.75654853620955, 83.55555555555556, 76.47058823529412, 100.0, 90.34482758620689, 92.63736263736266, 94.28571428571429, 100.0] [True, True, False, True, True, True, True, False, True, True]


In [22]:
count_true_positive = 0
for ver in verdict:
    if ver:
        count_true_positive +=1

In [23]:
count_true_positive

6244

In [18]:
import json
with open('matching_by_name_results_country_filter-1-2-3-4-5_clustered.json', 'w') as f:
    json.dump(result, f)
    f.close()

In [ ]:
# Find the best match
input_name = "venture nig vince"
best_match, best_score, top_matches = match_name(input_name, corpus)
print(f"Best match: {best_match} with score: {best_score}")

In [14]:
from rapidfuzz import fuzz, process
def sorted_tokens(name):
    return " ".join(sorted(name.split()))

# Matching
def match_name(input_name, corpus):
    input_name = clean_company_name(input_name)
    input_sorted = sorted_tokens(input_name)
    best_match = None
    best_score = 0
    for name in corpus:
        # Compute different similarity measures
        raw_score = fuzz.ratio(input_name, name)
        
        sorted_score = fuzz.ratio(input_sorted, sorted_tokens(name))

        # Combine scores
        combined_score = (raw_score + sorted_score) / 2
        
        if combined_score > best_score:
            best_score = combined_score
            best_match = name
    
    return best_match, best_score

# Find the best match
input_name = "venture nig vince"
best_match, best_score = match_name(input_name, corpus)
print(f"Best match: {best_match} with score: {best_score}")

Best match: vincee venture nig with score: 80.0


In [30]:
from rapidfuzz import fuzz, process
import heapq
def sorted_tokens(name):
    return " ".join(sorted(name.split()))

def levenshtein_matching_score(input_name, name):
    levenshtein_matching_score = fuzz.ratio(input_name, name)
    return levenshtein_matching_score

def token_set_matching_score(input_name, name):
    token_set_matching_score = fuzz.token_set_ratio(input_name, name)
    return token_set_matching_score

def partial_matching_score(input_name, name):
    partial_matching_score = fuzz.partial_ratio(input_name, name)
    return partial_matching_score

def substring_matching_score(input_name, name):
    substring_matching_score = fuzz.partial_ratio(input_name, name)
    return substring_matching_score

def token_sort_ratio_score(input_name, name):
    score = fuzz.token_sort_ratio(input_name, name)
    return score
# Matching
max_number_matches = 5
def match_name(input_name, corpus):
    input_name = clean_company_name(input_name)
    best_match = None
    best_score = 0
    top_matches = []
    heapq.heapify(top_matches)
    similarity_measures = { 
        "levenshtein_matching_score" : (1, levenshtein_matching_score),
        "token_set_matching_score" : (0, token_set_matching_score),
        "substring_matching_score" : (0, substring_matching_score),
        "partial_matching_score" : (0, partial_matching_score),
        "token_sort_ratio_score" : (1, token_sort_ratio_score)
    }
    for name in corpus:
        num_measures = 0
        total_score = 0
        for function_check in similarity_measures.values():
            check, func = function_check
            if check:
                score = func(input_name, name)
                total_score += score
                num_measures +=1
            if num_measures == 0:
                combined_score = 0
            else:
                combined_score = total_score / num_measures
        if len(top_matches) < max_number_matches:
            heapq.heappush(top_matches, (combined_score, name))
        else:
            heapq.heappushpop(top_matches, (combined_score, name))
        if combined_score > best_score:
            best_score = combined_score
            best_match = name
    
    return best_match, best_score, top_matches

# Find the best match
input_name = "venture nig vince"
best_match, best_score, top_matches = match_name(input_name, corpus)
print(f"Best match: {best_match} with score: {best_score}")

Best match: vincee venture nig with score: 80.0


In [31]:
top_matches

[(67.5, 'vidson ventures nigeria'),
 (67.56756756756756, 'zelidon ventures nig'),
 (70.27027027027026, 'sir vin ventures nig'),
 (68.57142857142857, 'zenas ventures nig'),
 (80.0, 'vincee venture nig')]

In [29]:
top_matches

[(70.27027027027027, 'zelidon ventures nig'),
 (70.47619047619048, 'zenas ventures nig'),
 (73.87387387387388, 'sir vin ventures nig'),
 (72.22222222222221, 'venture'),
 (85.71428571428571, 'vincee venture nig')]